In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3092755,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3092756,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3092757,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3092758,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


In [10]:
###
#cal_PMf  
###
u=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    u[i]=a.T

###
#obs_PMf
###
v=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_obs'][5112*i:5112*i+5112])
    v[i]=a.T

print(np.shape(u))
print(np.shape(v))

(544, 5112)
(544, 5112)


In [11]:
XRestruct_Fun=u
YRestruct_Fun=v
XCov_fun=np.cov(XRestruct_Fun.T)
XMean_fun=np.zeros(5112)
YMean_fun=np.zeros(5112)
for i in range(0,5112):
    for j in range(0,544):
        XMean_fun[i]=XMean_fun[i]+(XRestruct_Fun[j][i]/544)
        YMean_fun[i]=YMean_fun[i]+(YRestruct_Fun[j][i]/544)
print(type(XRestruct_Fun))
print(type(YRestruct_Fun))
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))
print(np.shape(XCov_fun))
print(np.shape(XMean_fun))
print(np.shape(YMean_fun))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(544, 5112)
(544, 5112)
(5112, 5112)
(5112,)
(5112,)


In [12]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xbar=XMean_fun
Ybar=YMean_fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))
print('Xbar')
#print(Xbar)
print(np.shape(Xbar))
print('Ybar')
#print(Ybar)
print(np.shape(Ybar))

Xhat
(544, 5112)
Yhat
(544, 5112)
Xbar
(5112,)
Ybar
(5112,)


In [13]:
G_xx=XCov_fun
G_xxt=np.zeros(5112)
for i in range(0,5112):
     G_xxt[i]=G_xx[i][i]
G_xy=0
for i in range (0,544):
    G_xyi=(Xhat[i]-Xbar)*(Yhat[i]-Ybar)
    G_xy=G_xy+G_xyi
G_xy=G_xy/543

print(G_xxt)
print(np.shape(G_xxt))
print(G_xy)
print(np.shape(G_xy))

[71.87628124 65.56788126 62.29095212 ... 26.01540284 23.70147909
 19.96894783]
(5112,)
[33.41140544 30.69395649 30.14079456 ...  9.54944286  9.75020169
  8.37133091]
(5112,)


In [14]:
B_1=G_xy/G_xxt
B_0=YMean_fun-(B_1*XMean_fun)

print('B_1')
#print(B_1)
print(np.shape(B_1))
print('B_0')
#print(B_0)
print(np.shape(B_0))

B_1
(5112,)
B_0
(5112,)


In [16]:
after_fx=np.zeros([605,5112])
for i in range (0,605):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    after_fx[i]=a.T
print(np.shape(after_fx))

(605, 5112)


In [17]:
fit_fx=B_0+(B_1*after_fx)
print(np.shape(fit_fx))

(605, 5112)


# Compare

In [20]:
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
data0

,SITE_ID,IH,TAU,FCST_TIME,POLLUTANT,VALUE,SITE_NAME_x,OBS_VALUE,SITE_NAME_y,AreaName,Township,County,address,lon,lat
0,EPA001,2020-02-24 08:00:00,0,2020-02-24 08:00:00,PM25,7.2521,基隆,14,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
1,EPA001,2020-02-24 08:00:00,1,2020-02-24 09:00:00,PM25,4.9510,基隆,10,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
2,EPA001,2020-02-24 08:00:00,2,2020-02-24 10:00:00,PM25,4.4674,基隆,13,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
3,EPA001,2020-02-24 08:00:00,3,2020-02-24 11:00:00,PM25,4.6159,基隆,11,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
4,EPA001,2020-02-24 08:00:00,4,2020-02-24 12:00:00,PM25,3.9937,基隆,11,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,68,2021-11-02 04:00:00,PM25,3.6190,關山,4,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135711,EPA080,2021-10-30 08:00:00,69,2021-11-02 05:00:00,PM25,3.7908,關山,7,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135712,EPA080,2021-10-30 08:00:00,70,2021-11-02 06:00:00,PM25,4.0454,關山,7,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135713,EPA080,2021-10-30 08:00:00,71,2021-11-02 07:00:00,PM25,3.9015,關山,4,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451


In [23]:
dd = data0.drop(data0[data0['TAU'] == 0].index)
dd

,SITE_ID,IH,TAU,FCST_TIME,POLLUTANT,VALUE,SITE_NAME_x,OBS_VALUE,SITE_NAME_y,AreaName,Township,County,address,lon,lat
1,EPA001,2020-02-24 08:00:00,1,2020-02-24 09:00:00,PM25,4.9510,基隆,10,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
2,EPA001,2020-02-24 08:00:00,2,2020-02-24 10:00:00,PM25,4.4674,基隆,13,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
3,EPA001,2020-02-24 08:00:00,3,2020-02-24 11:00:00,PM25,4.6159,基隆,11,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
4,EPA001,2020-02-24 08:00:00,4,2020-02-24 12:00:00,PM25,3.9937,基隆,11,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
5,EPA001,2020-02-24 08:00:00,5,2020-02-24 13:00:00,PM25,3.9602,基隆,9,基隆,北部空品區,信義區,基隆市,基隆市信義區東信路324號,121.7601,25.1292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135710,EPA080,2021-10-30 08:00:00,68,2021-11-02 04:00:00,PM25,3.6190,關山,4,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135711,EPA080,2021-10-30 08:00:00,69,2021-11-02 05:00:00,PM25,3.7908,關山,7,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135712,EPA080,2021-10-30 08:00:00,70,2021-11-02 06:00:00,PM25,4.0454,關山,7,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451
3135713,EPA080,2021-10-30 08:00:00,71,2021-11-02 07:00:00,PM25,3.9015,關山,4,關山,花東空品區,臺東縣,臺東縣,臺東縣關山鎮自強路66號,121.1619,23.0451


In [25]:
"""
old data
"""
old_x=np.zeros([605,5112])
old_y=np.zeros([605,5112])
#data_cal
for i in range (0,605):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i]=a
#data_obs
for i in range (0,605):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit_fx))

(605, 5112)
(605, 5112)
(605, 5112)


In [26]:
old_x1=np.zeros([544,5112])
old_y1=np.zeros([544,5112])
fit_fx1=np.zeros([544,5112])
for i in range (0,544):
    for j in range (0,5112):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit_fx[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
print("January to August (training data set) before_error (MADE) : %f" %before_error1)
print("January to August (training data set) after_error (MADE) : %f" %after_error1)

January to August (training data set) before_error (MADE) : 7.181135
January to August (training data set) after_error (MADE) : 6.295365


In [27]:
old_x2=np.zeros([61,5112])
old_y2=np.zeros([61,5112])
fit_fx2=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        old_x2[i][j]=old_x[i+544][j]
        old_y2[i][j]=old_y[i+544][j]
        fit_fx2[i][j]=fit_fx[i+544][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
print("September (testing data set) before_error (MADE) : %f" %before_error2)
print("September (testing data set) after_error (MADE) : %f" %after_error2)

September (testing data set) before_error (MADE) : 6.928633
September (testing data set) after_error (MADE) : 6.149006
